Environment setup.

In [ ]:
!pip install transformers
!pip install datasets
!pip install --upgrade accelerate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
colab_data_path = "/content/drive/MyDrive/ribe_512x768/"
colab_dir = "/content/drive/MyDrive/Seminar2/model/checkpoint-900/"
model_name_or_path = "google/vit-base-patch16-224-in21k"

Imports.

In [4]:
import numpy as np
import torch
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets, DatasetDict
from transformers import (
    ViTForImageClassification,
    ViTFeatureExtractor,
    Trainer,
    TrainingArguments,
)
from transformers.trainer_utils import EvalPrediction
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

Prepare dataset splits.

In [ ]:
train_dataset = load_dataset("imagefolder", data_dir=colab_data_path + '/train', split='train')
splits = train_dataset.train_test_split(test_size=0.33)
test_dataset = load_dataset("imagefolder", data_dir=colab_data_path + '/test', split='test')

In [6]:
dataset =DatasetDict({'train': splits['train'], 'val': splits['test'], 'test': test_dataset})

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 892
    })
    val: Dataset({
        features: ['image', 'label'],
        num_rows: 440
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 717
    })
})

Define image augmentations.

In [8]:
transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomRotation(30),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
])

Apply the transforms to the train set. Concatenate the transformed and original train set.

In [ ]:
transformed_train_dataset = dataset["train"].map(
    lambda example: {"image": transform(example["image"]), "label": example["label"]}
)

dataset["train"] = concatenate_datasets([transformed_train_dataset, dataset["train"]])

Configure the Feature Extractor and apply it to the dataset.

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained(
    model_name_or_path,
    do_resize=False,
    patch_size=64,
)

feature_extractor.image_mean = [0.485, 0.456, 0.406]
feature_extractor.image_std = [0.229, 0.224, 0.225]


def transform(example_batch):
    inputs = feature_extractor(
        [x.convert("RGB") for x in example_batch["image"]], return_tensors="pt"
      )
    inputs["labels"] = example_batch["label"]
    return inputs

dataset = dataset.with_transform(transform)

id2label = {id: label for id, label in enumerate(dataset["train"].features["label"].names)}
label2id = {label: id for id, label in id2label.items()}

Configure Vision Transformer model.

In [ ]:
# Set interpolate_pos_encoding=True in the source code.
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

model.config.hidden_dropout_prob = 0.5

Define auxiliary functions for the training procedure and configure training parameters.

In [12]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

def compute_metrics(eval_pred: EvalPrediction):
  preds = np.argmax(eval_pred.predictions, axis=1)
  return {
    "acc": accuracy_score(eval_pred.label_ids, preds),
    "f1": f1_score(eval_pred.label_ids, preds, average="weighted"),
    "precision": precision_score(eval_pred.label_ids, preds, average="weighted"),
    "recall": recall_score(eval_pred.label_ids, preds, average="weighted")
    }

In [14]:
training_args = TrainingArguments(
  output_dir=colab_dir,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=20,
  fp16=True,
  save_steps=60,
  eval_steps=60,
  warmup_steps=500,
  logging_steps=60,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    tokenizer=feature_extractor,
)

Train the model.

In [15]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall
60,0.966300,0.727031,0.745455,0.650726,0.594579,0.745455
120,0.530800,0.367115,0.881818,0.881824,0.890558,0.881818
180,0.387800,0.324624,0.854545,0.839243,0.887213,0.854545
240,0.255300,0.436802,0.836364,0.813636,0.881054,0.836364
300,0.233100,0.705457,0.754545,0.674413,0.810220,0.754545
360,0.303500,0.486438,0.779545,0.765586,0.880258,0.779545
420,0.318500,0.365781,0.863636,0.850149,0.893074,0.863636
480,0.353700,0.185133,0.934091,0.932851,0.937464,0.934091
540,0.226400,0.296619,0.897727,0.896237,0.908302,0.897727
600,0.190600,0.208247,0.918182,0.916313,0.922131,0.918182


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***** train metrics *****
  epoch                    =          20.0
  total_flos               = 20179984305GF
  train_loss               =        0.0756
  train_runtime            =    1:04:03.94
  train_samples_per_second =         9.282
  train_steps_per_second   =          1.16


Evaluate the model on valid.

In [16]:
metrics = trainer.evaluate(dataset["val"])
trainer.log_metrics("validation", metrics)
trainer.save_metrics("validation", metrics)

***** validation metrics *****
  epoch                   =       20.0
  eval_acc                =     0.9705
  eval_f1                 =     0.9704
  eval_loss               =     0.1068
  eval_precision          =     0.9704
  eval_recall             =     0.9705
  eval_runtime            = 0:00:16.75
  eval_samples_per_second =     26.261
  eval_steps_per_second   =      3.283


Evaluate the model on test.

In [17]:
metrics = trainer.evaluate(dataset["test"])
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

***** test metrics *****
  epoch                   =       20.0
  eval_acc                =     0.5132
  eval_f1                 =     0.5297
  eval_loss               =     1.8439
  eval_precision          =     0.6438
  eval_recall             =     0.5132
  eval_runtime            = 0:05:05.23
  eval_samples_per_second =      2.349
  eval_steps_per_second   =      0.295
